# 5. Calibration of single ended measurement with WLS and confidence intervals

A single ended calibration is performed with weighted least squares. Over all timesteps simultaneous. $\gamma$ and $\alpha$ remain constant, while $C$ varies over time. The weights are not considered equal here. The weights kwadratically decrease with the signal strength of the measured Stokes and anti-Stokes signals.

The confidence intervals can be calculated as the weights are correctly defined.

In [ ]:
import os

from dtscalibration import read_xml_dir
import matplotlib.pyplot as plt

In [ ]:
try:
    wd = os.path.dirname(os.path.realpath(__file__))
except:
    wd = os.getcwd()

filepath = os.path.join(wd, '..', '..', 'tests', 'data', 'single_ended')
timezone_netcdf = 'UTC',
timezone_ultima_xml = 'Europe/Amsterdam'
file_ext = '*.xml'

ds = read_xml_dir(filepath,
                  timezone_netcdf=timezone_netcdf,
                  timezone_ultima_xml=timezone_ultima_xml,
                  file_ext=file_ext)

ds = ds.sel(x=slice(-30, 101))  # only calibrate parts of the fiber
sections = {
            'probe1Temperature':    [slice(20, 25.5)],  # warm bath
            'probe2Temperature':    [slice(5.5, 15.5)],  # cold bath
#             'referenceTemperature': [slice(-24., -4)]  # The internal coil is not so uniform
            }
ds.sections = sections

In [ ]:
print(ds.calibration_single_ended.__doc__)

In [ ]:
ds.data_vars

In [ ]:
st_label = 'ST'
ast_label = 'AST'

First calculate the variance in the measured Stokes and anti-Stokes signals, in the forward and backward direction.

The Stokes and anti-Stokes signals should follow a smooth decaying exponential. This function fits a decaying exponential to each reference section for each time step. The variance of the residuals between the measured Stokes and anti-Stokes signals and the fitted signals is used as an estimate of the variance in measured signals.

In [ ]:
st_var, resid = ds.variance_stokes(st_label=st_label, suppress_info=1)
ast_var, _ = ds.variance_stokes(st_label=ast_label, suppress_info=1)

Similar to the ols procedure, we make a single function call to calibrate the temperature. If the method is `wls` and confidence intervals are passed to `conf_ints`, confidence intervals calculated. As weigths are correctly passed to the least squares procedure, the covariance matrix can be used. This matrix holds the covariances between all the parameters. A large parameter set is generated from this matrix, assuming the parameter space is normally distributed with their mean at the best estimate of the least squares procedure.

The large parameter set is used to calculate a large set of temperatures. By using `percentiles` or `quantile` the 95% confidence interval of the calibrated temperature between 2.5% and 97.5% are calculated.

The confidence intervals differ per time step. If you would like to calculate confidence intervals of all time steps together you have the option `ci_avg_time_flag=True`. 'We can say with 95% confidence that the temperature remained between this line and this line during the entire measurement period'.

In [ ]:
ds.calibration_single_ended(sections=sections,
                            st_label=st_label,
                            ast_label=ast_label,
                            st_var=st_var,
                            ast_var=ast_var,
                            method='wls',
                            # conf_ints=[0.00135, 0.025, 0.15865, 0.5, 0.84135, 0.975, 0.99865],
                            conf_ints=[0.025, 0.5, 0.975],
                            ci_avg_time_flag=0,
                            store_tempvar='_var',
                            conf_ints_size=500,
                            solver='stats')

Lets compare our calibrated values with the device calibration

In [ ]:
ds1 = ds.isel(time=0)  # take only the first timestep

ds1.TMPF.plot(linewidth=1, label='User calibrated')  # plot the temperature calibrated by us
ds1.TMP.plot(linewidth=1, label='Device calibrated')  # plot the temperature calibrated by the device
plt.title('Temperature at the first time step')
plt.legend()

In [ ]:
ds.TMPF_MC_var.plot()

In [ ]:
ds1.TMPF_MC.sel(CI=0.025).plot(label = '2.5% CI')
ds1.TMPF_MC.sel(CI=0.975).plot(label = '97.5% CI')
ds1.TMPF.plot(label='User calibrated')
plt.title('User calibrated temperature with 95% confidence interval')
plt.legend()

We can tell from the graph above that the 95% confidence interval widens furtherdown the cable. Lets have a look at the calculated variance along the cable for a single timestep. According to the device manufacturer this should be around 0.005929 degC.

In [ ]:
ds1.TMPF_MC_var.plot()

The variance of the temperature measurement appears to be larger than what the manufacturer reports. This is already the case for the internal cable; it is not caused by a dirty connector/bad splice on our side. Maybe the length of the calibration section was not sufficient.

At 30 m the variance sharply increases. There are several possible explanations. E.g., large temperatures or decreased signal strength.

Lets have a look at the Stokes and anti-Stokes signal.

In [ ]:
ds1.ST.plot()
ds1.AST.plot()

Clearly there was a bad splice at 30 m that resulted in the sharp increase of measurement uncertainty for the cable section after the bad splice.